## Load data

In [2]:
file_test = 'C:\Data\Miroculus\A3_1_2016_text_inputs\A3_1_2016_binary_data_entities_normalize_rmstopwords_stem_test.tsv'
file_train = 'C:\Data\Miroculus\A3_1_2016_text_inputs\A3_1_2016_binary_data_entities_normalize_rmstopwords_stem_train.tsv'

In [3]:
import pandas as pd
test = pd.read_csv(file_test, quotechar='"', delimiter='\t', header=None, names=['Label', 'Text'])
train = pd.read_csv(file_train, quotechar='"', delimiter='\t', header=None, names=['Label', 'Text'])
test.head; train.head

<bound method NDFrame.head of              Label                                               Text
0      NO_RELATION  result identifi previous report breast cancer-...
1      NO_RELATION  conclus taken togeth mmvarentty1mm/ggvarentty1...
2         RELATION  revers correl mmvarentty1mm ggvarentty1gg also...
3         RELATION  result suggest recognit site mmvarentty1mm mmv...
4         RELATION  issu jci ghosh colleagu identifi uniqu microrn...
5      NO_RELATION  conclus mmvarentty1mm may promot cell ggvarent...
6      NO_RELATION        howev role mmvarentty1mm npc remain unknown
7         RELATION  targetscan predict ggvarentty1gg ggvarentty2gg...
8      NO_RELATION  increas mmvarentty1mm depend upon ggvarentty1g...
9      NO_RELATION  revers correl mmvarentty1mm ggvarentty1gg expr...
10        RELATION  studi identifi mmvarentty1mm mmvarentty2mm mic...
11        RELATION  show mmvarentty1mm oncosuppress microrna lost ...
12     NO_RELATION  newli identifi mmvarentty1mm/ggvarentty1

## Vectorise text data into numerical vector

### Using `CountVectorizer`

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1,1), max_df=0.9, min_df=0.01)
train_tfidf_vectors = tfidf_vectorizer.fit_transform(train.Text)

** Note: ** Numbers not removed. Consider removing numbers.

`tfidf_vectorizer.vocabulary_` is a `dict`

In [5]:
tfidf_vectorizer.vocabulary_

{'001': 0,
 '05': 1,
 '16': 2,
 '3p': 3,
 '5p': 4,
 'act': 5,
 'activ': 6,
 'addit': 7,
 'affect': 8,
 'aim': 9,
 'alter': 10,
 'analys': 11,
 'analysi': 12,
 'analyz': 13,
 'apoptosi': 14,
 'arrest': 15,
 'assay': 16,
 'associ': 17,
 'bind': 18,
 'bioinformat': 19,
 'biomark': 20,
 'blot': 21,
 'breast': 22,
 'cancer': 23,
 'carcinoma': 24,
 'caus': 25,
 'cell': 26,
 'chain': 27,
 'chang': 28,
 'clinic': 29,
 'cluster': 30,
 'colon': 31,
 'combin': 32,
 'compar': 33,
 'conclus': 34,
 'confirm': 35,
 'contribut': 36,
 'control': 37,
 'correl': 38,
 'crc': 39,
 'cycl': 40,
 'data': 41,
 'decreas': 42,
 'demonstr': 43,
 'depend': 44,
 'detect': 45,
 'determin': 46,
 'develop': 47,
 'differ': 48,
 'differenti': 49,
 'direct': 50,
 'directli': 51,
 'diseas': 52,
 'downregul': 53,
 'downstream': 54,
 'ectop': 55,
 'effect': 56,
 'elev': 57,
 'enhanc': 58,
 'epitheli': 59,
 'evalu': 60,
 'evid': 61,
 'examin': 62,
 'express': 63,
 'factor': 64,
 'famili': 65,
 'final': 66,
 'format': 67,
 'f

In [6]:
train_tfidf_vectors.shape

(10783, 233)

In [7]:
train_tfidf_vectors.data

array([ 0.35076054,  0.30204213,  0.21158038, ...,  0.37578529,
        0.28848477,  0.42515082])

In [8]:
print(train_tfidf_vectors.nnz) # number of non-zero elements
print(train_tfidf_vectors.nnz / float(train_tfidf_vectors.shape[0])) # non-zero per sample
print(train_tfidf_vectors.nnz / float(train_tfidf_vectors.shape[0] * train_tfidf_vectors.shape[1]) * 100) # non-zero per feature space in %

101851
9.44551609014189
4.053869566584503


## Data Preprocessing: 
### Attributes and Target

### Convert categorical labels into numerical labels

In [9]:
from sklearn import preprocessing

In [10]:
le = preprocessing.LabelEncoder()
le.fit(train.Label)
list(le.classes_)

['NO_RELATION', 'RELATION']

In [11]:
y_train = le.transform(train.Label);
y_test = le.transform(test.Label);

print(y_train[0:5])
print(le.inverse_transform(y_train[0:5]))

[0 0 1 1 1]
['NO_RELATION' 'NO_RELATION' 'RELATION' 'RELATION' 'RELATION']


`X` is data, `y` is target. `X` has the size of `n_samples` x `n_features`, `y` has `n_samples` x 1 labels

In [12]:
X_train = train_tfidf_vectors

In [13]:
X_test = tfidf_vectorizer.transform(test.Text)

In [14]:
X_test.shape

(3595, 233)

In [15]:
X_test.data

array([ 0.2597683 ,  0.27758534,  0.27108252, ...,  0.18050815,
        0.36407148,  0.19773178])

## Log Res: 

In [16]:
from sklearn import linear_model
clf = linear_model.LogisticRegression(C=1e5, class_weight='balanced')
clf.fit(X_train, y_train)

LogisticRegression(C=100000.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [17]:
y_pred = clf.predict(X_test)

In [18]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.73      0.73      0.73      1804
          1       0.73      0.72      0.72      1791

avg / total       0.73      0.73      0.73      3595



In [19]:
print(metrics.confusion_matrix(y_test, y_pred))

[[1316  488]
 [ 498 1293]]


### Coeficients of the decision function The top word

In [20]:
clf.coef_

array([[  5.31609855e-01,   6.61359273e-02,   2.23516114e+00,
          4.67947281e-01,  -8.85513455e-01,   6.06873961e-01,
          8.81714179e-01,   9.48469114e-01,   5.33088181e-01,
         -3.09098139e-01,   9.65049662e-01,   4.08373339e-01,
          8.97055393e-02,   6.28302212e-01,   3.75398890e-01,
          3.18007584e-01,   4.60677670e-01,  -4.03836024e-01,
          1.34551133e+00,   1.84675085e+00,  -1.03648445e+00,
          2.69441216e+00,   3.97349803e-01,  -1.14898713e+00,
         -6.98510193e-01,   1.33292470e+00,  -2.54747665e-02,
          6.44022027e-01,   4.19774604e-01,   1.01787670e-02,
          1.17258740e+00,   1.12856293e+00,   1.09243192e+00,
         -6.69362469e-01,   6.04148163e-01,   1.02918960e+00,
          7.20562049e-01,   7.01783847e-01,   7.64146246e-01,
          3.15932820e-01,   1.74100595e+00,   2.98447887e-01,
          1.03223196e+00,   4.07250601e-01,   1.24692338e+00,
          3.35259497e-01,   1.27231563e+00,  -1.09308002e-01,
        

In [21]:
len(clf.coef_[0,])

233

Get the index of coef with the maximum value.

In [22]:
clf.coef_.argmax()

73

In [23]:
m = clf.coef_.argmax()
print(list(tfidf_vectorizer.vocabulary_.keys())[list(tfidf_vectorizer.vocabulary_.values()).index(m)])

ggvarentty1gg


Get the index of the highest 10 coef value.

**Note** `clf.coef_` is ndarray, see below:

In [24]:
clf.coef_[0]

array([  5.31609855e-01,   6.61359273e-02,   2.23516114e+00,
         4.67947281e-01,  -8.85513455e-01,   6.06873961e-01,
         8.81714179e-01,   9.48469114e-01,   5.33088181e-01,
        -3.09098139e-01,   9.65049662e-01,   4.08373339e-01,
         8.97055393e-02,   6.28302212e-01,   3.75398890e-01,
         3.18007584e-01,   4.60677670e-01,  -4.03836024e-01,
         1.34551133e+00,   1.84675085e+00,  -1.03648445e+00,
         2.69441216e+00,   3.97349803e-01,  -1.14898713e+00,
        -6.98510193e-01,   1.33292470e+00,  -2.54747665e-02,
         6.44022027e-01,   4.19774604e-01,   1.01787670e-02,
         1.17258740e+00,   1.12856293e+00,   1.09243192e+00,
        -6.69362469e-01,   6.04148163e-01,   1.02918960e+00,
         7.20562049e-01,   7.01783847e-01,   7.64146246e-01,
         3.15932820e-01,   1.74100595e+00,   2.98447887e-01,
         1.03223196e+00,   4.07250601e-01,   1.24692338e+00,
         3.35259497e-01,   1.27231563e+00,  -1.09308002e-01,
         4.55571153e-01,

In [25]:
print(clf.coef_[0][0])
print(clf.coef_[0][1])

0.531609855221
0.0661359272512


In [26]:
import numpy as np
topN = np.argsort(clf.coef_[0])[::-1][:20]
print(topN)

[ 73  74 211 100  21 167 148   2 181 231  50 130 224 112  98 192  19 185
 176  53]


In [27]:
for v in topN:
    print(list(tfidf_vectorizer.vocabulary_.keys())[list(tfidf_vectorizer.vocabulary_.values()).index(v)])


ggvarentty1gg
ggvarentty2gg
target
invers
blot
protein
partial
16
repress
western
direct
mmvarentty5mm
untransl
luciferas
interact
rt
bioinformat
restor
regul
downregul


In [28]:
import numpy as np
bottomN = np.argsort(clf.coef_[0])[::1][:20]
print(bottomN)

[170 189 151 205 150  23  83 132  20  52 140 122   4 118 134 108  24  87
  33 196]


In [29]:
for v in bottomN:
    print(list(tfidf_vectorizer.vocabulary_.keys())[list(tfidf_vectorizer.vocabulary_.values()).index(v)])


reaction
risk
pcr
stage
patient
cancer
healthi
model
biomark
diseas
non
mice
5p
metastasi
molecular
liver
carcinoma
human
compar
serum
